### Experimentation

In [11]:
import numpy as np
import pandas as pd
from wordfreq import word_frequency
from wordfreq import zipf_frequency
import math

Experimentation with the zipf_frequency versus the word_frequency functions.

You can achieve similar results by setting val = 1 / rate when rate is word_frequency()

In [44]:
words = ["sweater", "the", "correct", "and", "cute", "fluffy"]

for i in words:
    rate = zipf_frequency(i, 'en')
    val = math.pow(10, 9 - rate)
    print(i, "is used approximately once per", str(val), "words in the English language")

sweater is used approximately once per 165958.6907437563 words in the English language
the is used approximately once per 18.620871366628656 words in the English language
correct is used approximately once per 16218.100973589299 words in the English language
and is used approximately once per 38.904514499428046 words in the English language
cute is used approximately once per 22387.21138568338 words in the English language
fluffy is used approximately once per 257039.57827688646 words in the English language


In [6]:
valid = np.loadtxt("valid-wordle-words.txt", dtype = str)

list = []
for i in valid:
    list.append(zipf_frequency(i, 'en'))

df = pd.DataFrame({"words": valid, "f-score": list})

### Testing

In [1]:
from predictor import Predictor
from predictor import Hint
import numpy as np
import pandas as pd

In [ ]:
wordle = Predictor() 
wordle.run_game_interface()